In [ ]:
pip install python-twitter

In [1]:
import twitter
import os
from config import *
import time

# initialize api instance
twitter_api = twitter.Api(consumer_key=consumer_key,
                         consumer_secret=consumer_secret,
                         access_token_key=access_token_key,
                         access_token_secret=access_token_secret,
                        cache=None,tweet_mode='compat')

#print(consumer_key,consumer_secret,access_token_key,access_token_secret)

# test authentication
print(twitter_api.VerifyCredentials())
# print(twitter_api.GetStatus(126402758403305000))

{"created_at": "Wed Jul 20 02:34:27 +0000 2016", "description": "I am Al and I am passionate about technology.", "followers_count": 5, "friends_count": 52, "id": 755591669194514432, "id_str": "755591669194514432", "location": "Tustin, CA", "name": "Al", "profile_background_color": "000000", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_banner_url": "https://pbs.twimg.com/profile_banners/755591669194514432/1469041661", "profile_image_url": "http://pbs.twimg.com/profile_images/755843006062694401/2JikSpJq_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/755843006062694401/2JikSpJq_normal.jpg", "profile_link_color": "1B95E0", "profile_sidebar_border_color": "000000", "profile_sidebar_fill_color": "000000", "profile_text_color": "000000", "screen_name": "alknowstech", "status": {"created_at": "Mon Jul 31 21:38:46 +0000 2017", 

In [2]:
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100,
                                               lang='en',result_type='recent')
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None, "user":status.user.screen_name, "time":status.created_at,
                "search_term":search_keyword} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [3]:
# create blank testDataSet list
testDataSet = []

In [25]:
# test search function
# repeat execution to pull more data or other keywords
search_term = input("Enter a search keyword: ")
testDataSet.extend(buildTestSet(search_term))

Enter a search keyword: arbys
Fetched 100 tweets for the term arbys


In [5]:
print(testDataSet[0:110])

[{'text': 'RT @_ErinTanner: Ahhh yes, I will light the onion candle to be reminded of a middle school gym locker room. Thanks McDonalds!', 'label': None, 'user': 'rachelleevilleg', 'time': 'Sat Feb 22 18:21:23 +0000 2020', 'search_term': 'mcdonalds'}, {'text': '@AlbertsStuff @JakeJayingee McDonalds', 'label': None, 'user': 'pooplolaka', 'time': 'Sat Feb 22 18:21:14 +0000 2020', 'search_term': 'mcdonalds'}, {'text': 'RT @The_X_Button: This is like choosing between mcdonalds and a fine lobster https://t.co/eQICfvSGyK', 'label': None, 'user': 'cesar_l01', 'time': 'Sat Feb 22 18:21:14 +0000 2020', 'search_term': 'mcdonalds'}, {'text': '@McDonalds Yes you are.', 'label': None, 'user': 'allnewtpir', 'time': 'Sat Feb 22 18:21:13 +0000 2020', 'search_term': 'mcdonalds'}, {'text': 'RT @pdxalerts: Portland &gt; police to @McDonalds (NE Grand), report of two employees physically fighting, may be related to one of them "smok…', 'label': None, 'user': 'starr3333', 'time': 'Sat Feb 22 18:21:06 +0000

# Use only to test twitter api

In [ ]:
tweets_fetched = twitter_api.GetSearch('disney', count = 100, lang='en')
[{"text":status.text, "label":None, "user":status.user.screen_name} for status in tweets_fetched]

# Start preprocessing of training data

In [6]:
import pandas as pd
import numpy as np

traindata140 = os.path.join(".","trainingandtestdata_sentiment140","training.csv")

In [7]:
#read the csv file command
train_df = pd.read_csv(traindata140, header=None, usecols=[0,5], names=['polarity of the tweet','text'], 
                       encoding="ISO-8859-1")
train_df.head()
# (0 = negative, 2 = neutral, 4 = positive)

,polarity of the tweet,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


# Use only to process the test dataset provided by Sentiment140

In [ ]:
testdata140 = os.path.join(".","trainingandtestdata_sentiment140","test.csv")
test_df = pd.read_csv(testdata140, header=None, usecols=[0,5],
                      names=['polarity of the tweet', 'text'],encoding="ISO-8859-1")
conditions_test = [
    (test_df['polarity of the tweet'] == 0),
    (test_df['polarity of the tweet'] == 2),
    (test_df['polarity of the tweet'] == 4)]
choices_test = ['negative', 'neutral', 'positive']
test_df['label'] = np.select(conditions_test, choices_test)
test_df.head()

test_ls = test_df.to_dict('records')

# Use to convert numeric value to Positive or Negative

In [8]:
train_df_negative = train_df.loc[train_df['polarity of the tweet']==0]
train_df_positive = train_df.loc[train_df['polarity of the tweet']==4]

In [9]:
# Pull evenly the amount of training set that will be used and combine them
frames = [train_df_negative.iloc[0:12500,:],train_df_positive.iloc[0:12500,:]]
result = pd.concat(frames)

In [10]:
conditions = [
    (result['polarity of the tweet'] == 0),
    (result['polarity of the tweet'] == 2),
    (result['polarity of the tweet'] == 4)]
choices = ['negative', 'neutral', 'positive']
result['label'] = np.select(conditions, choices)
result.head()

,polarity of the tweet,text,label
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negative
1,0,is upset that he can't update his Facebook by ...,negative
2,0,@Kenichan I dived many times for the ball. Man...,negative
3,0,my whole body feels itchy and like its on fire,negative
4,0,"@nationwideclass no, it's not behaving at all....",negative


# Convert dataframe to dictionary to run analysis

In [11]:
train_ls = result.to_dict('records')

# Function to clean up the dataset

In [12]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import nltk
# nltk.download('punkt')

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]
    
tweetProcessor = PreProcessTweets()

In [13]:
# Process Training Set
preprocessedTrainingSet = tweetProcessor.processTweets(train_ls)

In [26]:
# Process Test Set
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

# Function to process NLTK

In [15]:
import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [16]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [17]:
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

# Execute only if model has to be retrained

In [ ]:
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

# SKLearnClassifier = SklearnClassifier(BernoulliNB()).train(trainingFeatures)
SKLearnClassifier = SklearnClassifier(SVC(), sparse=False).train(trainingFeatures)

# Compare the test set with the model

In [ ]:
SKLearnResultLabels = [SKLearnClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# Save the model to pickle for later use

In [ ]:
# Save Pickle
import pickle
save_classifier = open("sklearn_svc_train_25k.pickle","wb")
pickle.dump(SKLearnClassifier, save_classifier)
save_classifier.close()

# Use to load pickle

In [18]:
# Load Pickle
import pickle
classifier_f = open("sklearn_svc_train_25k.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

# Compare the test set with the loaded pickle

In [27]:
SKResultLabelsPickle = [classifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# Export results

In [28]:
# had to add space before tweet text to eliminate csv issue with tweets that begins with @
# list comprehension
test_list_text = [' '+test['text'] for test in testDataSet]
# convert twitter time output to real time output
test_list_time = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(test['time'],'%a %b %d %H:%M:%S +0000 %Y')) 
for test in testDataSet]
test_list_search_term = [test['search_term'] for test in testDataSet]

In [29]:
output_list = pd.DataFrame(
    {'Tweet': test_list_text,
     'Time': test_list_time,
     'Result': SKResultLabelsPickle,
     'Search_Term': test_list_search_term
    })

In [30]:
output_list['Score']=np.where((output_list.Result=='negative'), -1, 1)

In [31]:
output_list

,Tweet,Time,Result,Search_Term,Score
0,"RT @_ErinTanner: Ahhh yes, I will light the o...",2020-02-22 18:21:23,positive,mcdonalds,1
1,@AlbertsStuff @JakeJayingee McDonalds,2020-02-22 18:21:14,positive,mcdonalds,1
2,RT @The_X_Button: This is like choosing betwe...,2020-02-22 18:21:14,positive,mcdonalds,1
3,@McDonalds Yes you are.,2020-02-22 18:21:13,positive,mcdonalds,1
4,RT @pdxalerts: Portland &gt; police to @McDon...,2020-02-22 18:21:06,negative,mcdonalds,-1
5,@RonnieOttey Of course. https://t.co/ZsrHJDDS50,2020-02-22 18:21:01,positive,mcdonalds,1
6,Why didn’t she say “ma’am this is a mcdonalds...,2020-02-22 18:21:00,positive,mcdonalds,1
7,Calum really was McDonalds #1 fan change my m...,2020-02-22 18:20:59,positive,mcdonalds,1
8,@SacredGeoInt We would have to assume that we...,2020-02-22 18:20:50,positive,mcdonalds,1
9,RT @TheSocietyDude: Bernie reminds me of my f...,2020-02-22 18:20:32,positive,mcdonalds,1


# Output to CSV file

In [27]:
output_list.to_csv('./outputSKLearn_test.csv',header=True,
                              index=False,encoding="utf-8")